In [1]:
import laspy
import sys
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
import rasterio
from rasterio.transform import from_origin
import os
import time
import math
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.morphology import grey_dilation, grey_opening
from scipy.ndimage import generate_binary_structure

In [2]:
MIN_X_COOR_INDEX = 0
MAX_X_COOR_INDEX = 1
MIN_Y_COOR_INDEX = 2
MAX_Y_COOR_INDEX = 3
MAX_INT = sys.maxsize

X = 0
Y = 1
H = 2
PX = 3
PY = 4
P = 3

In [3]:
#filename_las_in = "../../data/lidar/thd_000029.las"
filename_las_in = "/home/roberto/Documents/LIDAR_DATA/THD26/thd_000026.las"
path_output = "../../data/lidar/thd_000026/"
try:  
    os.mkdir(path_output)
except OSError:  
    print ("Creation of the directory %s failed" % path_output)
else:  
    print ("Successfully created the directory %s " % path_output)

Successfully created the directory ../../data/lidar/thd_000026/ 


In [4]:
def get_classified_point_cloud(classification):
    in_file = laspy.file.File(filename_las_in, mode = "r")
    #out_file = open(sys.argv[2]+".txt", 'w')
    point_records = in_file.points.copy()

    print("Class = "+str(classification))
    trees_only = np.where(in_file.raw_classification == classification)

    # pull out full point records of filtered points, and create an XYZ array for KDTree
    trees_points = in_file.points[trees_only]

    
    array_x = []
    array_y = []
    array_z = []
    #roi_xy = [MAX_INT, 0, MAX_INT, 0]
    
    for trees_point in trees_points:
        #print("%.2f %.2f" % (trees_point['point']['X'], trees_point['point']['Y']))
        array_x.append(trees_point['point']['X']/100)
        array_y.append(trees_point['point']['Y']/100)
        array_z.append(trees_point['point']['Z']/100)
        #out_file.write("%.2f %.2f %.2f\n" % (trees_point['point']['X']/100, trees_point['point']['Y']/100, trees_point['point']['Z']/100))
    array_x = np.array(array_x)
    array_y = np.array(array_y)  
    array_z = np.array(array_z)
    raster_roi = [np.min(array_x), np.max(array_x), np.min(array_y), np.max(array_y)]
    return array_x, array_y, array_z, raster_roi


In [ ]:
def save_raster(x_coor, y_coor, array, res,filename_out):
    print("save_raster")
    array = np.flip(array, axis=0)
    transform = from_origin(x_coor, y_coor, res, res)
    chm_raster = rasterio.open(filename_out, 'w', driver='GTiff',
                            height = array.shape[0], width = array.shape[1],
                            count=1, dtype=str(array.dtype),
                            crs='EPSG:32638',
                            transform=transform)
    chm_raster.write(array, 1)
    chm_raster.close()
    

### Create raster for DTM

In [ ]:
x_array_ground, y_array_ground, z_array_ground, roi_ground = get_classified_point_cloud(2)

# data coordinates and values
x = np.array(x_array_ground)
y = np.array(y_array_ground)
z = np.array(z_array_ground)

# target grid to interpolate to
xi = np.arange(roi_ground[MIN_X_COOR_INDEX],roi_ground[MAX_X_COOR_INDEX],1)
yi = np.arange(roi_ground[MIN_Y_COOR_INDEX],roi_ground[MAX_Y_COOR_INDEX],1)
xi,yi = np.meshgrid(xi,yi)

# interpolate
zi = griddata((x,y),z,(xi,yi),method='linear')
print("-----------------------------------------------------------")
filename_dtm = path_output+"dtm.tif"
save_raster(roi_ground[MIN_X_COOR_INDEX], roi_ground[MAX_Y_COOR_INDEX],zi,1,filename_dtm)



Class = 2


### Create CHM raster dimension

In [ ]:
file_dtm_in = rasterio.open(filename_dtm)
dtm_band = file_dtm_in.read(1)

RES = 0.33 # 40cm

x_array_surface, y_array_surface, z_array_surface, roi_surface = get_classified_point_cloud(5)

raster_coor = [roi_ground[MIN_X_COOR_INDEX], 
               roi_ground[MAX_Y_COOR_INDEX],
               roi_ground[MAX_X_COOR_INDEX] - roi_ground[MIN_X_COOR_INDEX],
               roi_ground[MAX_Y_COOR_INDEX] - roi_ground[MIN_Y_COOR_INDEX]]  # [x0, y0, width, height]

raster_size = [int(math.ceil(raster_coor[2]/RES)), int(math.ceil(raster_coor[3]/RES))]  # [num_cols, num_rows]

print(raster_coor)
print(raster_size)

In [ ]:
file_dtm_in = rasterio.open(path_output+"dtm.tif")
dtm_band = file_dtm_in.read(1)


array = []
for i in range(0, len(x_array_surface)):
    values = [x_array_surface[i], y_array_surface[i], z_array_surface[i]]
    # Need to read the raster with geographic coordinates instead
    row, col = file_dtm_in.index(x_array_surface[i], y_array_surface[i])
    dtm = dtm_band[row, col]
    if dtm >= 0:
        values[H] -= dtm   #Mean value of DTM
    else:
        values[H] = 0
    pixel_coor_x = math.floor((values[X]-raster_coor[X]) / raster_coor[2] * raster_size[X])
    #pixel_coor_y = raster_size[Y] - math.floor((values[Y]-raster_coor[Y]) / raster_coor[3] * raster_size[Y])
    pixel_coor_y = raster_size[Y] - math.floor((raster_coor[Y]-values[Y]) / raster_coor[3] * raster_size[Y])
    values.append(pixel_coor_x)
    values.append(pixel_coor_y)
    array.append(values)

In [ ]:
chm_array = np.full((raster_size[Y], raster_size[X]), 0.0)

for point in array:
    if point[PX] < raster_size[X] and point[PY] < raster_size[Y]:
        if point[H] > chm_array[int(point[PY]),int(point[PX])]:
            chm_array[int(point[PY]),int(point[PX])] = point[H]
footprint = generate_binary_structure(2, 1)
filtered_chm = grey_dilation(chm_array, footprint=footprint)
#filtered_chm = gaussian_filter(chm_array, sigma=1)
#filtered_chm = chm_array
chm_array = np.maximum(filtered_chm, chm_array)
save_raster(raster_coor[X], raster_coor[Y],filtered_chm, RES, path_output+"chm.tif")